<img src="http://www.cs.wm.edu/~rml/images/wm_horizontal_single_line_full_color.png">

<h1 style="text-align:center;">CSCI 420/520: Introduction to Machine Learning final exam, Fall 2017</h1>
<h1 style="text-align:center;">Problem 1</h1>

<h1 style="text-align:center;">Due by 1700 on Tuesday, December 11, 2017</h1>

# Name: Yang Song

# The CIFAR10 dataset

For this problem you will use the [CIFAR10 dataset](https://www.cs.toronto.edu/~kriz/cifar.html).  This consists of 32 x 32 images of 10 different classes; there are 50,000 training cases and 10,000 test cases.

# What you are to do

* Create a CNN that will perform image classification.
    * Train it with the CIFAR10 training data and evaluate it with the CIFAR10 test data.
    * Your model should have at least one convolutional layer (my best model had four, but I had success with fewer).
    * If you perform a lot of tuning, you should only submit your best model.
* Populate this Jupyter notebook with the code needed to create, train, and run your CNN.
    * As part of your model evaluation, your notebook should compute and print the confusion matrix and f1-scores.
* You should submit your CNN stored as the HDF5 file <code>problem1.hdf5</code> (see below).
* Your model should achieve &ge; 50% accuracy on the test set (I reached 50% in six epochs with a very simple model, and 67% in 10 epochs with a deeper network).  If your accuracy is around 10%, which would be the expected result of random guessing, then something is seriously amiss.
* High accuracy is not of paramount importance for this problem; the goal of this question is whether you understand how to build and train the network and can explain what your network does and its structure.

# Questions you are to answer about your model

Include your answers in this cell.

1.  Explain which each part of your model does.  E.g., if you include a convolutional layer or a dropout stage, explain what goes on in that stage, and the stage's purpose (e.g., downsampling). I used several convolutional layers. Two consisted of 32 filters, each 3 x 3, and two consisted of 64 filters, each 3 x 3. They apply convolution operations to the input and pass the results to the next layer. This is for dealing with data in smaller grids, allowing more efficiency, while adding more layers could improve accuracy. I also used several dropout stages, just to make sure the model is not overfitted.

2.  Explain the number of model parameters returned by the <code>summary()</code> method of the <code>Sequential</code> class. Layer parameters are shown in output. These are all trainable parameters

3.  How many epochs of training did you use? I used 6 epochs. 

4.  Which classes are most confused with one another?  Least confused?  Explain whether these results make sense to you. The most confused ones are: classes 1 and 9, classes 2 and 10. One of the least confused is between 5 and 10. 


<br/><br/>

# Reading the data and scaling the data

You should use the Keras interface to this dataset.  The first time you read the training and test sets Keras will download the data.

**Very important!  Use the following block of code for scaling and label encoding and no other!  Do not perform any feature extraction such as PCA, either.  This is how I will test your model.**

In [1]:
import keras
from keras.datasets import cifar10

# There are 10 classes of images.
num_classes = 10

# The data, split between train and test sets.
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Scale the data.
x_train = x_train.astype('float32')
x_test  = x_test.astype('float32')
x_train /= 255
x_test  /= 255

# Convert class vectors to binary class matrices.
y_train_enc = keras.utils.to_categorical(y_train, num_classes)
y_test_enc  = keras.utils.to_categorical(y_test,  num_classes)

print("Data read and scaled!")

Using TensorFlow backend.


Data read and scaled!


<br/><br/>

# Your code follows&hellip;

# Define the network

In [2]:
from keras import models, layers, losses, optimizers
import tensorflow as tf
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, Activation, Flatten




print('constructing the model...', end='')

model = models.Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

print('done!')

opt = optimizers.rmsprop(lr=0.0001, decay=1e-6)
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])


constructing the model...done!


# Count the model parameters

Assuming your model is named <code>model</code>&hellip;

In [3]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 64)        18496     
__________

# Train the model

In [4]:
batch_size = 128
epochs = 6

model.fit(x_train, y_train_enc,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test_enc))
print("Done training the model!")

Train on 50000 samples, validate on 10000 samples
Epoch 1/6
50000/50000 [==============================] - 283s 6ms/step - loss: 2.0002 - acc: 0.2670 - val_loss: 1.7406 - val_acc: 0.3873
Epoch 2/6
50000/50000 [==============================] - 279s 6ms/step - loss: 1.6965 - acc: 0.3882 - val_loss: 1.5705 - val_acc: 0.4366
Epoch 3/6
50000/50000 [==============================] - 277s 6ms/step - loss: 1.5663 - acc: 0.4324 - val_loss: 1.4414 - val_acc: 0.4747
Epoch 4/6
50000/50000 [==============================] - 279s 6ms/step - loss: 1.4734 - acc: 0.4681 - val_loss: 1.3672 - val_acc: 0.5083
Epoch 5/6
50000/50000 [==============================] - 279s 6ms/step - loss: 1.4089 - acc: 0.4928 - val_loss: 1.3181 - val_acc: 0.5311
Epoch 6/6
50000/50000 [==============================] - 294s 6ms/step - loss: 1.3577 - acc: 0.5131 - val_loss: 1.2582 - val_acc: 0.5498
Done training the model!


In [5]:
import numpy as np
from sklearn import metrics

def compute_metrics (classifier, X_test, y_test):
    """
    This function computes and prints various performance measures for a classifier.
    """
    # Use the classifier to make predictions for the test set.
    y_pred = classifier.predict(X_test)
    
    # Choose the class with the highest estimated probability.
    y_pred = np.argmax(y_pred, axis=1)


    # Compute the confusion matrix.
    cm = metrics.confusion_matrix(y_test, y_pred)
    print('Confusion matrix, without normalization')
    print(cm, '\n')

    # Normalize the confusion matrix by row (i.e by the number of samples in each class).
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    np.set_printoptions(precision=3, linewidth=132)
    print('Normalized confusion matrix')
    print(cm_normalized, '\n')

    # The confusion matrix as percentages.
    cm_percentage = 100 * cm_normalized
    print('Confusion matrix as percentages')
    print(np.array2string(cm_percentage, formatter={'float_kind':lambda x: "%6.2f" % x}), '\n')
    
    # Precision, recall, and f-score.
    print(metrics.classification_report(y_test, y_pred, digits=3))

    return cm

In [6]:
compute_metrics(model, x_test, y_test)


Confusion matrix, without normalization
[[590  27  43  28  23  10  22  27 164  66]
 [ 34 650   5   9   2   2  28  25  65 180]
 [ 93  16 265  76 207  79 124  84  28  28]
 [ 28   9  45 331  81 180 162 106  19  39]
 [ 36   8  53  64 466  38 162 144  17  12]
 [ 13   6  62 178  63 450  72 129  17  10]
 [  8   4  28  54  73  16 745  47   5  20]
 [ 16   2  19  57  64  73  43 675  10  41]
 [118  58  11  20   6   8  11  19 688  61]
 [ 44 121   8  10   7   6  48  56  62 638]] 

Normalized confusion matrix
[[ 0.59   0.027  0.043  0.028  0.023  0.01   0.022  0.027  0.164  0.066]
 [ 0.034  0.65   0.005  0.009  0.002  0.002  0.028  0.025  0.065  0.18 ]
 [ 0.093  0.016  0.265  0.076  0.207  0.079  0.124  0.084  0.028  0.028]
 [ 0.028  0.009  0.045  0.331  0.081  0.18   0.162  0.106  0.019  0.039]
 [ 0.036  0.008  0.053  0.064  0.466  0.038  0.162  0.144  0.017  0.012]
 [ 0.013  0.006  0.062  0.178  0.063  0.45   0.072  0.129  0.017  0.01 ]
 [ 0.008  0.004  0.028  0.054  0.073  0.016  0.745  0.047  0.

array([[590,  27,  43,  28,  23,  10,  22,  27, 164,  66],
       [ 34, 650,   5,   9,   2,   2,  28,  25,  65, 180],
       [ 93,  16, 265,  76, 207,  79, 124,  84,  28,  28],
       [ 28,   9,  45, 331,  81, 180, 162, 106,  19,  39],
       [ 36,   8,  53,  64, 466,  38, 162, 144,  17,  12],
       [ 13,   6,  62, 178,  63, 450,  72, 129,  17,  10],
       [  8,   4,  28,  54,  73,  16, 745,  47,   5,  20],
       [ 16,   2,  19,  57,  64,  73,  43, 675,  10,  41],
       [118,  58,  11,  20,   6,   8,  11,  19, 688,  61],
       [ 44, 121,   8,  10,   7,   6,  48,  56,  62, 638]])

# Evaluate the model

In [7]:
score = model.evaluate(x_test, y_test_enc, verbose=0)
print('Test loss:    ', score[0])
print('Test accuracy:', score[1])
print("Done evaluating the model!")

Test loss:     1.25822776012
Test accuracy: 0.5498
Done evaluating the model!


# Save the model

Make you sure you get the filename right, as in the following block!

In [8]:
# Save the trained model (assuming the variable name is "model").
print('Saving the model...', end='')
model.save('problem1.hdf5')
print('done!')

Saving the model...done!
